<a href="https://colab.research.google.com/github/brunoodon/4_workshop_misp/blob/main/Hunting_automatizado_de_CVE_recentemente_exploradas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
#instalando o módulo PyMISP
!pip3 install pymisp

In [42]:
#importando as bibliotecas necessárias
import pandas as pd
import requests
import numpy as np
import csv
import json
import datetime
import time
import statistics
from pandas.io.parsers import python_parser
from pymisp import MISPEvent, MISPObject, PyMISP, ExpandedPyMISP, MISPSharingGroup
#url dos exploits do CISA
url_cisa = 'https://www.cisa.gov/sites/default/files/csv/known_exploited_vulnerabilities.csv'
#url do inventário de ativos do cliente
url_inventory = 'https://raw.githubusercontent.com/brunoodon/4_workshop_misp/main/assets.csv'
#url do MISP
url_misp = 'https://misp.brunoodon.com.br'
#authkey de usuário habilitado para criar eventos no MISP
misp_key = '73RhcpLzFq38Ts5vLO0xhlcF4zEbRFJih8iRiKdw'
#url do dataset de vulnerabilidades recentemente exploradas
#url do Elastic
url_es = 'http://mvp-puc.brunoodon.com.br:9200/alerts/_doc'

In [43]:
#lendo e dando 'print' no dataset do inventário do ativos do cliente
inventory = pd.read_csv(url_inventory)
display(inventory)

,application,version,ip,hostname,dep
0,Exchange Server,2016,172.16.0.20,mail.fakecomp.com,TI
1,Apache HTTP Server,2.4,172.16.0.21,www.fakecomp.com,TI
2,Apache HTTP Server,2.4.50,172.16.0.21,soar.fakecom.com,Segurança Cibernética
3,Chrome,10.0,10.0.0.152,desktop04,Vendas
4,Chrome,9.0,192.168.1.12,desktop20,Laboratório
5,Chrome,12.0,192.168.7.34,dp04,RH
6,Chromium,10.0,10.0.0.150,desktop01,Vendas
7,Chromium,10.0,10.0.0.151,desktop02,Vendas
8,Chromium,9.0,10.0.0.152,desktop04,Vendas
9,ESXi,6.0,10.30.2.200,virt-02,TI


In [44]:
#lendo e dando 'print' no dataset de vulnerabilidades com exploit recente
exploited_cve = pd.read_csv(url_cisa)
display(exploited_cve)

,cveID,vendorProject,product,vulnerabilityName,dateAdded,shortDescription,requiredAction,dueDate,notes
0,CVE-2021-27104,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,NaN
1,CVE-2021-27102,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,NaN
2,CVE-2021-27101,Accellion,FTA,Accellion FTA SQL Injection Vulnerability,2021-11-03,Accellion FTA contains a SQL injection vulnera...,Apply updates per vendor instructions.,2021-11-17,NaN
3,CVE-2021-27103,Accellion,FTA,Accellion FTA Server-Side Request Forgery (SSR...,2021-11-03,Accellion FTA contains a server-side request f...,Apply updates per vendor instructions.,2021-11-17,NaN
4,CVE-2021-21017,Adobe,Acrobat and Reader,Adobe Acrobat and Reader Heap-based Buffer Ove...,2021-11-03,Acrobat Acrobat and Reader contain a heap-base...,Apply updates per vendor instructions.,2021-11-17,NaN
...,...,...,...,...,...,...,...,...,...
975,CVE-2023-29298,Adobe,ColdFusion,Adobe ColdFusion Improper Access Control Vulne...,2023-07-20,Adobe ColdFusion contains an improper access c...,Apply mitigations per vendor instructions or d...,2023-08-10,https://helpx.adobe.com/security/products/cold...
976,CVE-2023-38205,Adobe,ColdFusion,Adobe ColdFusion Improper Access Control Vulne...,2023-07-20,Adobe ColdFusion contains an improper access c...,Apply mitigations per vendor instructions or d...,2023-08-10,https://helpx.adobe.com/security/products/cold...
977,CVE-2023-35078,Ivanti,Endpoint Manager Mobile,Ivanti Endpoint Manager Mobile Authentication ...,2023-07-25,"Ivanti Endpoint Manager Mobile (EPMM, previous...",Apply mitigations per vendor instructions or d...,2023-08-15,https://forums.ivanti.com/s/article/CVE-2023-3...
978,CVE-2023-38606,Apple,Multiple Products,Apple Multiple Products Kernel Unspecified Vul...,2023-07-26,"Apple iOS, iPadOS, macOS, tvOS, and watchOS co...",Apply mitigations per vendor instructions or d...,2023-08-16,"https://support.apple.com/en-us/HT213841, http..."


In [ ]:
#percorrendo cada linha do dataset de inventário de ativos do cliente
for a in range(len(inventory)):
  application=inventory.values[a][0]
  version = inventory.values[a][1]
  ip=inventory.values[a][2]
  hostname=inventory.values[a][3]
  dep=inventory.values[a][4]
  #criando uma variável com a data atual para inserirmos como parte do nome do evento
  today=str(datetime.date.today())
  #fazendo a busca dos registros do dataset de vulnerabilidades exploradas onde o campo 'Product' contém o nome da aplicação descrita em cada linha de inventário
  search = exploited_cve.query('product.str.contains("'+application+'") == True', engine="python")
  misp_verifycert = False
  #definindo as características do evento do MISP
  misp = ExpandedPyMISP(url_misp, misp_key, misp_verifycert)
  event = MISPEvent()
  event.info = f"Risco de vulnerabilidades no ativo {hostname} | Aplicação: {application} - {version} - "+today+""
  event.analysis = "2"
  event.published = True
  event.distribution = "4" #compartilhar com Sharing group
  event.sharing_group_id = "1" #ID do Sharing group que receberá o evento
  event.threat_level_id = "1" #level HIGH
  event.add_tag('Exploited_CVE')#uma tag criada para identificar que este evento tem CVE com exploit
  event.add_tag('tlp:amber+strict') #só a entidade que enviou e a entidade que recebeu podem ter acesso ao avento
  event.add_tag('circl:incident-classification="vulnerability"')#tag que identifica o tipo de ameaça que será compartilhada. Neste caso, uma tag da taxonomia do CIRCL.
  #percorrendo cada resultado dessa busca e atribuíndo os valores às variáveis
  for b in range(len(search)):
      cve=search.values[b][0]
      vendor=search.values[b][1]
      product=search.values[b][2]
      vulnerability_name=search.values[b][3]
      date_added=search.values[b][4]
      short_desc=search.values[b][5]
      due_date=search.values[b][6]
      notes=search.values[b][7]
      #fazendo a consulta do CVE no banco do NVD, para enriquecer o dataset
      url_nvd = 'https://services.nvd.nist.gov/rest/json/cves/2.0?cveId='+cve+''
      r_nvd = requests.get(url_nvd)
      try:
        #lendo o json com o resultado e atribuindo cada resultado de campo às varáveis severity_score e vector_string
        #obs: foi preciso estabelecer 2 possíveis estruturas, uma para cada versão da métrica de severity score (V2 ou V3.1)
        json_nvd = json.loads(r_nvd.text)
        for c in json_nvd['vulnerabilities']:
          print(c)
          cpe = []
          try:
            for d in c['cve']['metrics']['cvssMetricV31']:
              vector_string = d['cvssData']['vectorString']
              #convertendo o campo de severity_score em float
              severity_score = float(d['cvssData']['baseScore'])
          except:
            for d in c['cve']['metrics']['cvssMetricV2']:
              vector_string = d['cvssData']['vectorString']
              severity_score = float(d['cvssData']['baseScore'])
          for b in c['cve']['configurations']:
            for x in b['nodes']:
              for z in x['cpeMatch']:
                #verificando se a versão da aplicação foi encontrada em algum CPE referente à vulnerabilidade
                if version in z['criteria']:
                  #se for o caso, inserir o CPE na lista
                  cpe.append(z['criteria'])
          #só adiciona a vulnerabilidade no Elastic e no MISP caso o CPE não seja vazio
          if cpe != []:
            #adicionando as CVE, com os detalhes no comentário
            event.add_attribute('vulnerability', str(cve), disable_correlation=True, to_ids=False, comment=f'Severity Score: {severity_score}|Vector String:{vector_string}|Product: {product}')
            print(f'Adicionando a CVE {cve} no evento')
            es_header = {'Content-Type': 'application/json'}
            es_json = {
            'Vulnerability': cve,
            'VulnerabilityName': vulnerability_name,
            'Description': short_desc,
            'IP': ip,
            'CPE': cpe,
            'Vendor': vendor,
            'Hostname': hostname,
            'Application': f'{application}:{version}',
            'ExploitationVector': vector_string,
            'SeverityScore': float(severity_score),
            'AlertDate': today,
            'Department': dep
            }
            r_es = requests.post(url_es, headers=es_header, verify=False, json=es_json)
            #foi necessário um temporizador porque a API do NVD tem limitação de requests por minuto
            time.sleep(8)
      except:
        print('Nenhuma vulnerabilidade encontrada no ativo')
  event = misp.add_event(event)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.brunoodon.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.brunoodon.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.brunoodon.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'cve': {'id': 'CVE-2020-17144', 'sourceIdentifier': 'secure@microsoft.com', 'published': '2020-12-10T00:15:16.120', 'lastModified': '2022-07-12T17:42:04.277', 'vulnStatus': 'Analyzed', 'cisaExploitAdd': '2021-11-03', 'cisaActionDue': '2022-05-03', 'cisaRequiredAction': 'Apply updates per vendor instructions.', 'cisaVulnerabilityName': 'Microsoft Exchange Server Remote Code Execution Vulnerability', 'descriptions': [{'lang': 'en', 'value': 'Microsoft Exchange Remote Code Execution Vulnerability This CVE ID is unique from CVE-2020-17117, CVE-2020-17132, CVE-2020-17141, CVE-2020-17142.'}, {'lang': 'es', 'value': 'Vulnerabilidad de ejecución de código remota en Microsoft Exchange Este ID de CVE es diferente de CVE-2020-17117, CVE-2020-17132, CVE-2020-17141, CVE-2020-17142'}], 'metrics': {'cvssMetricV31': [{'source': 'nvd@nist.gov', 'type': 'Primary', 'cvssData': {'version': '3.1', 'vectorString': 'CVSS:3.1/AV:N/AC:L/PR:H/UI:R/S:C/C:H/I:H/A:H', 'attackVector': 'NETWORK', 'attackComplexity'

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.brunoodon.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.brunoodon.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.brunoodon.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/c

{'cve': {'id': 'CVE-2020-16010', 'sourceIdentifier': 'chrome-cve-admin@google.com', 'published': '2020-11-03T03:15:15.603', 'lastModified': '2020-11-04T18:51:15.703', 'vulnStatus': 'Analyzed', 'cisaExploitAdd': '2021-11-03', 'cisaActionDue': '2022-05-03', 'cisaRequiredAction': 'Apply updates per vendor instructions.', 'cisaVulnerabilityName': 'Google Chrome for Android Heap Buffer Overflow Vulnerability', 'descriptions': [{'lang': 'en', 'value': 'Heap buffer overflow in UI in Google Chrome on Android prior to 86.0.4240.185 allowed a remote attacker who had compromised the renderer process to potentially perform a sandbox escape via a crafted HTML page.'}, {'lang': 'es', 'value': 'Un desbordamiento del búfer de la pila en UI de Google Chrome en Android anterior a versión 86.0.4240.185, permitió que un atacante remoto que había comprometido el proceso de renderizado pudiera realizar un escape del sandbox por medio de una página HTML diseñada'}], 'metrics': {'cvssMetricV31': [{'source': '

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.brunoodon.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.brunoodon.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.brunoodon.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/c

...

{'cve': {'id': 'CVE-2019-11634', 'sourceIdentifier': 'cve@mitre.org', 'published': '2019-05-22T17:29:00.227', 'lastModified': '2020-08-24T17:37:01.140', 'vulnStatus': 'Analyzed', 'cisaExploitAdd': '2021-11-03', 'cisaActionDue': '2022-05-03', 'cisaRequiredAction': 'Apply updates per vendor instructions.', 'cisaVulnerabilityName': 'Citrix Workspace Application and Receiver for Windows Remote Code Execution Vulnerability', 'descriptions': [{'lang': 'en', 'value': 'Citrix Workspace App before 1904 for Windows has Incorrect Access Control.'}, {'lang': 'es', 'value': 'La aplicación Citrix Workspace antes de 1904 para Windows tiene un control de acceso incorrecto.'}], 'metrics': {'cvssMetricV30': [{'source': 'nvd@nist.gov', 'type': 'Primary', 'cvssData': {'version': '3.0', 'vectorString': 'CVSS:3.0/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H', 'attackVector': 'NETWORK', 'attackComplexity': 'LOW', 'privilegesRequired': 'NONE', 'userInteraction': 'NONE', 'scope': 'UNCHANGED', 'confidentialityImpact': '

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.brunoodon.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.brunoodon.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.brunoodon.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/c

{'cve': {'id': 'CVE-2019-11634', 'sourceIdentifier': 'cve@mitre.org', 'published': '2019-05-22T17:29:00.227', 'lastModified': '2020-08-24T17:37:01.140', 'vulnStatus': 'Analyzed', 'cisaExploitAdd': '2021-11-03', 'cisaActionDue': '2022-05-03', 'cisaRequiredAction': 'Apply updates per vendor instructions.', 'cisaVulnerabilityName': 'Citrix Workspace Application and Receiver for Windows Remote Code Execution Vulnerability', 'descriptions': [{'lang': 'en', 'value': 'Citrix Workspace App before 1904 for Windows has Incorrect Access Control.'}, {'lang': 'es', 'value': 'La aplicación Citrix Workspace antes de 1904 para Windows tiene un control de acceso incorrecto.'}], 'metrics': {'cvssMetricV30': [{'source': 'nvd@nist.gov', 'type': 'Primary', 'cvssData': {'version': '3.0', 'vectorString': 'CVSS:3.0/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H', 'attackVector': 'NETWORK', 'attackComplexity': 'LOW', 'privilegesRequired': 'NONE', 'userInteraction': 'NONE', 'scope': 'UNCHANGED', 'confidentialityImpact': '

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.brunoodon.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.brunoodon.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.brunoodon.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/c

Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontrada no ativo
Nenhuma vulnerabilidade encontra